# 2.3. 自动求梯度
在深度学习中，我们经常需要对函数求梯度（gradient）。本节将介绍如何使用 Tensorflow 提供的 GradientTape() 模块来自动求梯度。

In [1]:
import tensorflow as tf 

## 2.3.1. 简单例子
我们先看一个简单例子：对函数  $y = 2x^Tx$  求关于列向量  x  的梯度。我们先创建变量x，并赋初值。

In [2]:
x = tf.reshape(tf.range(4, dtype=tf.float32), [4,1])
x 

<tf.Tensor: id=8, shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [2.],
       [3.]], dtype=float32)>

In [3]:
with tf.GradientTape() as t:
    t.watch(x)
    y = 2 * tf.matmul(tf.transpose(x), x)
    
# 使用 tape 计算 y 关于 x 的梯度
dy_dx = t.gradient(y, x)
dy_dx

<tf.Tensor: id=28, shape=(4, 1), dtype=float32, numpy=
array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)>

函数 $y = 2x^Tx$  关于 x  的梯度应为 4x 。现在我们来验证一下求出来的梯度是正确的。

In [4]:
assert tf.norm((dy_dx - 4*x)).numpy() == 0.0 

## 2.3.3. 对Python控制流求梯度
即使函数的计算图包含了Python的控制流（如条件和循环控制），Tensorflow 也有可能对变量求梯度。

In [5]:
def f(x,y):
    output = 1.0 
    for i in range(y):
        if i > 1 and i < 5: 
            output = tf.multiply(output, x)
    return output

def grad(x,y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x,y)
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0 

tensorflow 还可以用来计算高阶梯度 

In [6]:
x = tf.Variable(1.0) # 创建一个初值为1.0的变量

with tf.GradientTape() as t: 
    with tf.GradientTape() as t2: 
        y = x*x*x
        # 在梯度 t 中计算 t2 对 x 的梯度
    dy_dx = t2.gradient(y,x)
d2y_dx2 = t.gradient(dy_dx, x)
    
assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0   

注：以上两个例子来源于 tensorflow [官方教程](https://www.tensorflow.org/tutorials/customization/autodiff)